In [1]:
class State:
    def __init__(self, left, right, umbrella, time_elapsed):
        self.left = left[:]
        self.right = right[:]
        self.umbrella = umbrella
        self.time = time_elapsed

    def is_goal(self):
        return len(self.left) == 0 and self.umbrella == 'R'

    def generate_moves(self):
        children = []
        current_side = self.left if self.umbrella == 'L' else self.right

        for i in range(len(current_side)):
            for j in range(i, len(current_side)):
                p1 = current_side[i]
                p2 = current_side[j]
                crossing = [p1] if i == j else [p1, p2]

                new_left = self.left[:]
                new_right = self.right[:]
                new_side = 'R' if self.umbrella == 'L' else 'L'

                if self.umbrella == 'L':
                    for p in crossing:
                        new_left.remove(p)
                        new_right.append(p)
                else:
                    for p in crossing:
                        new_right.remove(p)
                        new_left.append(p)

                new_time = self.time + max(crossing)
                child = State(new_left, new_right, new_side, new_time)
                children.append(child)

        return children

    def __eq__(self, other):
        return isinstance(other, State) and \
               sorted(self.left) == sorted(other.left) and \
               sorted(self.right) == sorted(other.right) and \
               self.umbrella == other.umbrella

    def __hash__(self):
        return hash((tuple(sorted(self.left)), tuple(sorted(self.right)), self.umbrella))


def get_name(time):
    names = {
        5: "Amogh",
        10: "Ameya",
        20: "Grandma",
        25: "Grandpa"
    }
    return names.get(time, f"Person({time})")


def remove_seen(children, open_list, closed_list):
    open_nodes = [n for n, _ in open_list]
    closed_nodes = [n for n, _ in closed_list]
    return [child for child in children if child not in open_nodes and child not in closed_nodes]


def reconstruct_path(goal_pair, closed_list):
    parent_map = {node: parent for node, parent in closed_list}
    node, parent = goal_pair
    path = [(node, parent)]
    while parent is not None:
        grandparent = parent_map[parent]
        path.append((parent, grandparent))
        parent = grandparent
    path.reverse()

    print("\nSolution Path:\n")

    for i, (current, prev) in enumerate(path):
        if prev is None:
            print(f"Step {i}: Start")
        else:
            moved_from = prev.left if prev.umbrella == 'L' else prev.right
            moved_people = list(set(moved_from) - set(current.left if prev.umbrella == 'L' else current.right))
            direction = "Right" if prev.umbrella == 'L' else "Left"
            names = [get_name(x) for x in moved_people]
            action = f"{' and '.join(names)} cross to {direction}"
            print(f"Step {i}: {action}")
        print(f"  Left     : {[get_name(p) for p in sorted(current.left)]}")
        print(f"  Right    : {[get_name(p) for p in sorted(current.right)]}")
        print(f"  Umbrella : {'Left' if current.umbrella == 'L' else 'Right'}")
        print(f"  Time     : {current.time} min\n")

    print("Goal reached")
    print(f"Total Time Taken: {path[-1][0].time} minutes")


def bfs(start_state):
    open_list = [(start_state, None)]
    closed_list = []

    while open_list:
        node, parent = open_list.pop(0)
        if node.is_goal():
            print("Goal Found using BFS")
            reconstruct_path((node, parent), closed_list)
            return
        closed_list.append((node, parent))
        children = node.generate_moves()
        new_children = remove_seen(children, open_list, closed_list)
        open_list += [(child, node) for child in new_children]
    print("No solution found using BFS")


def dfs(start_state):
    open_list = [(start_state, None)]
    closed_list = []

    while open_list:
        node, parent = open_list.pop(0)
        if node.is_goal():
            print("Goal Found using DFS")
            reconstruct_path((node, parent), closed_list)
            return
        closed_list.append((node, parent))
        children = node.generate_moves()
        new_children = remove_seen(children, open_list, closed_list)
        open_list = [(child, node) for child in new_children] + open_list
    print("No solution found using DFS")


Amogh = 5
Ameya = 10
Grandma = 20
Grandpa = 25

initial_state = State(
    left=[Amogh, Ameya, Grandma, Grandpa],
    right=[],
    umbrella='L',
    time_elapsed=0
)

print("BFS SOLUTION")
bfs(initial_state)
print("\nDFS SOLUTION")
dfs(initial_state)


BFS SOLUTION
Goal Found using BFS

Solution Path:

Step 0: Start
  Left     : ['Amogh', 'Ameya', 'Grandma', 'Grandpa']
  Right    : []
  Umbrella : Left
  Time     : 0 min

Step 1: Ameya and Amogh cross to Right
  Left     : ['Grandma', 'Grandpa']
  Right    : ['Amogh', 'Ameya']
  Umbrella : Right
  Time     : 10 min

Step 2: Amogh cross to Left
  Left     : ['Amogh', 'Grandma', 'Grandpa']
  Right    : ['Ameya']
  Umbrella : Left
  Time     : 15 min

Step 3: Grandpa and Grandma cross to Right
  Left     : ['Amogh']
  Right    : ['Ameya', 'Grandma', 'Grandpa']
  Umbrella : Right
  Time     : 40 min

Step 4: Ameya cross to Left
  Left     : ['Amogh', 'Ameya']
  Right    : ['Grandma', 'Grandpa']
  Umbrella : Left
  Time     : 50 min

Step 5: Ameya and Amogh cross to Right
  Left     : []
  Right    : ['Amogh', 'Ameya', 'Grandma', 'Grandpa']
  Umbrella : Right
  Time     : 60 min

Goal reached
Total Time Taken: 60 minutes

DFS SOLUTION
Goal Found using DFS

Solution Path:

Step 0: Start
  